# Import

In [6]:
import pandas as pd
import numpy as np
from astropy.io import fits
from forecast_SN_GW import get_hubblefit

# Load data

In [7]:
d_ps1 = pd.read_csv('../Data/SN_ps_snls_sdss_hst.csv', sep=' ', index_col='CID')
ps1_pd = d_ps1[d_ps1['IDSURVEY'] == 15]

d_X = pd.read_csv('../Data/jla_lcparams.txt', sep=' ', index_col='#name')

bias_data = fits.getdata('../Data/covmat/C_bias.fits')
cal_data = fits.getdata('../Data/covmat/C_cal.fits')
dust_data = fits.getdata('../Data/covmat/C_dust.fits')
host_data = fits.getdata('../Data/covmat/C_host.fits')
model_data = fits.getdata('../Data/covmat/C_model.fits')
nonia_data = fits.getdata('../Data/covmat/C_nonia.fits')
pecvel_data = fits.getdata('../Data/covmat/C_pecvel.fits')
stat_data = fits.getdata('../Data/covmat/C_stat.fits')
covX = bias_data + cal_data + dust_data + host_data + model_data + nonia_data + pecvel_data + stat_data

# Reproduce JLA results

## Build data array for JLA fit

In [ ]:
mass = np.zeros_like(d_X['3rdvar'].values)
for i, m in enumerate(d_X['3rdvar'].values):
    if m > 10:
        mass[i] = -1.
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, mass])

d_sigma = pd.read_csv('../Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## JLA fit

In [4]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])

In [5]:
g.fit()

/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: errordef is not given. Default to 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter Mb is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha1 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha2 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha3 is floating but does not have init

FCN = 682.891153089 
 TOTAL NCALL = 159 
 NCALLS = 159 
 
 
 EDM = 1.82717284962e-05 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,Mb,-19.0485,0.0231703,0,0,,,
2,alpha1,0.141227,0.00659009,0,0,,,
3,alpha2,-3.10154,0.080728,0,0,,,
4,alpha3,-0.0699574,0.0231108,0,0,,,
5,omgM,0.295371,0.0339336,0,0,,,


(0.15154930258417332, 0.0050877430812842316)

# Fit with linear correction in redshift 

## Build data array 

In [8]:
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, d_X['zcmb'].values])

d_sigma = pd.read_csv('../Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## Fit 

In [9]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])
g.fit()

/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: errordef is not given. Default to 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter Mb is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha1 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha2 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/florian/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha3 is floating but does not have init

FCN = 691.236130433 
 TOTAL NCALL = 204 
 NCALLS = 204 
 
 
 EDM = 5.18119666697e-05 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,Mb,-19.0682,0.023213,0,0,,,
2,alpha1,0.136253,0.00637933,0,0,,,
3,alpha2,-3.10188,0.0806027,0,0,,,
4,alpha3,-0.222551,0.193861,0,0,,,
5,omgM,0.469795,0.17517,0,0,,,


(0.15357536940078587, 0.005000199022698167)